### Here very simple single layer Autoencoder is Implemented. The main goal of Autoencoder is to produce the outputs same as given inputs. Usually, we are willing to create the outputs from the nosy input data, but here we didn't add any noise into data to make the understanding easy.

In [1]:
import tensorflow as tf
from util.data import *

/home/akkasi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


#### Parameters and Data ---------------------------------------------------------------------------------------------------------

In [9]:
Sample_number=1000
Feature_Number=1024
batch_size=256
learning_rate=0.01
display_step=20
epochs=100
hidden_units=500
DataObj=Data(data_len=Feature_Number,Sample_Number=Sample_number,Test_percentage=0)# we don't need test data
Input,Train_y,_,_=DataObj.CNN_data()
Number_of_batches=int(len(Input)/batch_size)
Output=Input

print(Input.shape)
print(Output.shape)


(1000, 1024)
(1000, 1024)


#### Structure definition ------------------------------------------------------------------------------------------------------

In [6]:
X=tf.placeholder(dtype=tf.float32,shape=[None,Feature_Number])
Y=tf.placeholder(dtype=tf.float32,shape=[None,Feature_Number])

Weights={'Wh':tf.Variable(tf.random_normal([Feature_Number,hidden_units],stddev=0.01)),
        'Wo':tf.Variable(tf.random_normal([hidden_units,Feature_Number],stddev=0.01))}
biases={'bh':tf.Variable(tf.random_normal([hidden_units],stddev=0.01)),
        'bo':tf.Variable(tf.random_normal([Feature_Number],stddev=0.01))}

#### Model Definition-------------------------------------------------------------------------------------------------------------------

In [18]:
def AutoEncoder(Inputs,Weights,biases):
    h=tf.nn.relu(tf.add(tf.matmul(Inputs,Weights['Wh']),biases['bh']))
    Outputs=tf.nn.tanh(tf.add(tf.matmul(h,Weights['Wo']),biases['bo']))
    return Outputs,h # h is the final results that we want!

In [42]:
Outputs,h=AutoEncoder(X,Weights,biases)
cost=tf.reduce_mean(tf.square(Outputs-Y))
train_step=tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
cross_entropy=-tf.reduce_sum(Y*tf.log(Outputs))

#### Graph Running -----------------------------------------------------------------------------------------------------------------

In [51]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
        avg_cost=0
        for j in range(Number_of_batches):
            Trx,_=next_batch(batch_size, Input,Train_y)
            
            sess.run(train_step,feed_dict={X:Trx,Y:Trx})
            avg_cost+=sess.run(cost,feed_dict={X:Trx,Y:Trx})/Number_of_batches
            
       # if i%display_step==0:
           # print ("Epoch: %03d/%03d cost: %.9f cross entropy: %.9f " 
                 #  % (i, epochs, avg_cost,ce))
    x=sess.run([Outputs,h],feed_dict={X:Input}) 
    
    print(sess.run(cross_entropy,feed_dict={X:tf.convert_to_tensor(Input,dtype=tf.float32),Y:tf.convert_to_tensor(Input,dtype=tf.float32)}))
    #print(x[0].shape,x[1].shape)
    
    
    
            
    

TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles.